## Batting Data Cleaning

In [61]:
import pandas as pd

In [62]:
batting_df = pd.read_csv('F:\DA Projects\CWC2023\ICC_CWC_2023_Data_Analysis\Data\pre_batting_df.csv')
batting_df.head(3)

,Unnamed: 0,Match_id,Score,Team,Opposite_Team,Innings
0,0,1,Bairstow c Daryl Mitchell b Santner 33 35 ...,England,New Zealand,1
1,1,1,Malan c Latham b Matt Henry 14 24 2 0 58.33,England,New Zealand,1
2,2,1,Root b Glenn Phillips 77 86 4 1 89.53,England,New Zealand,1


In [63]:
batting_df.drop(columns=['Unnamed: 0'],inplace=True)

In [64]:
batting_df.head()

,Match_id,Score,Team,Opposite_Team,Innings
0,1,Bairstow c Daryl Mitchell b Santner 33 35 ...,England,New Zealand,1
1,1,Malan c Latham b Matt Henry 14 24 2 0 58.33,England,New Zealand,1
2,1,Root b Glenn Phillips 77 86 4 1 89.53,England,New Zealand,1
3,1,Harry Brook c Conway b Rachin Ravindra 25 ...,England,New Zealand,1
4,1,Moeen b Glenn Phillips 11 17 1 0 64.71,England,New Zealand,1


In [65]:
def extract_scores_to_df(each_batsman_sample):
    '''
    This Function will Extract Stats of Batsman Statitcs such as Runs,Balls,Boundaries and Sixes (Numericals)
    Input : Batsman Stats as a row 
    Output : Batsman Stats as df 

    Sample Input : 'Bairstow    c Daryl Mitchell b Santner  33 35 4 1 94.29'
    '''
    stc = each_batsman_sample.split(' ')[-5:]  # only last 5 characters are Numericals see above cell for approach 
    
    # Extracting each value from scores assign it to appropriate attributes
    return pd.Series({
        'Runs': stc[0],
        'Balls': stc[1],
        'Boundaries': stc[2],
        'Sixes': stc[3],
        'StrikeRate': stc[4]
    })

In [66]:
batting_stats = batting_df['Score'].apply(extract_scores_to_df)

In [67]:
batting_stats.head()

,Runs,Balls,Boundaries,Sixes,StrikeRate
0,33,35,4,1,94.29
1,14,24,2,0,58.33
2,77,86,4,1,89.53
3,25,16,4,1,156.25
4,11,17,1,0,64.71


In [68]:
batting_df.shape,batting_stats.shape

((876, 5), (876, 5))

In [69]:
# Concatenating 2 dfs row_wise to get exact score card (Dismissal+Stats)
updated_batting_df = pd.concat([batting_df,batting_stats],axis=1)  
updated_batting_df

,Match_id,Score,Team,Opposite_Team,Innings,Runs,Balls,Boundaries,Sixes,StrikeRate
0,1,Bairstow c Daryl Mitchell b Santner 33 35 ...,England,New Zealand,1,33,35,4,1,94.29
1,1,Malan c Latham b Matt Henry 14 24 2 0 58.33,England,New Zealand,1,14,24,2,0,58.33
2,1,Root b Glenn Phillips 77 86 4 1 89.53,England,New Zealand,1,77,86,4,1,89.53
3,1,Harry Brook c Conway b Rachin Ravindra 25 ...,England,New Zealand,1,25,16,4,1,156.25
4,1,Moeen b Glenn Phillips 11 17 1 0 64.71,England,New Zealand,1,11,17,1,0,64.71
...,...,...,...,...,...,...,...,...,...,...
871,48,Travis Head c Shubman Gill b Siraj 137 120...,Australia,India,2,137,120,15,4,114.17
872,48,Mitchell Marsh c Rahul b Bumrah 15 15 1 1 ...,Australia,India,2,15,15,1,1,100.00
873,48,Steven Smith lbw b Bumrah 4 9 1 0 44.44,Australia,India,2,4,9,1,0,44.44
874,48,Marnus Labuschagne not out 58 110 4 0 52.73,Australia,India,2,58,110,4,0,52.73


In [70]:
data_list = []
def batsman_stats(sample):
    '''
    This will Create Separate columns for Each attribute in Batsman Score 
    such as Batsman, Dismissal Type,Bowler,Fielder,Bowler

    Input : Score Column, Output : Details of Score not numericals
    Sample Input : 'Bairstow    c Daryl Mitchell b Santner  33 35 4 1 94.29'
    '''
    for itr in sample:
        score = itr.split(' ')[:-6]  # upto Numericals last 6 characters are Numericals so upto 6 characters
        data_dict = {} # For Storing Each attribute in dict

        # for any dismissal type : Batsman | Dismissal Type | Fielder | Bowler 

        # Caught and Bowled : 'and' only present in caught and bowled
        if 'and' in score:
            data_dict['Batsman'] = " ".join(score[:score.index('c')]).strip() # upto c in 'caught'
            data_dict['Dismissal Type'] = 'Caught and Bowled' 
            data_dict['Fielder'] = '-' # No Fielder 
            data_dict['Bowler'] = " ".join(score[score.index('b')+1:]).strip() # for bowler 'b'


        # if 'c' and 'b' present in string then it is caught 
        elif 'c' in score and 'b' in score:
            data_dict['Batsman'] = " ".join(score[:score.index('c')]).strip() # upto c in 'caught'
            data_dict['Dismissal Type'] = 'Catch'
            # Fielder Name will come before Bowler 
            data_dict['Fielder'] = " ".join(score[score.index('c')+1:score.index('b')]).strip() 
            data_dict['Bowler'] = " ".join(score[score.index('b')+1:]).strip() # for bowler 'b'

        # stump
        elif 'st' in score and 'b' in score:        
            data_dict['Batsman'] = " ".join(score[:score.index('st')]).strip() 
            data_dict['Dismissal Type'] = 'Stump'  
            data_dict['Fielder'] = " ".join(score[score.index('st')+1:score.index('b')]).strip()
            data_dict['Bowler'] = " ".join(score[score.index('b')+1:]).strip()

        # lbw 
        elif 'lbw' in score and 'b' in score:        
            data_dict['Batsman'] = " ".join(score[:score.index('lbw')]).strip()
            data_dict['Dismissal Type'] = 'lbw'
            data_dict['Fielder'] = '-'
            data_dict['Bowler'] = " ".join(score[score.index('b')+1:]).strip()

        # Bowled
        elif 'b' in score:        
            data_dict['Batsman'] = " ".join(score[:score.index('b')]).strip()
            data_dict['Dismissal Type'] = 'Bowled'
            data_dict['Fielder'] = '-'
            data_dict['Bowler'] = " ".join(score[score.index('b')+1:]).strip()

        # Not out 
        elif 'not' in score:        
            data_dict['Batsman'] = " ".join(score[:score.index('not')]).strip()
            data_dict['Dismissal Type'] = 'Not out'
            data_dict['Fielder'] = '-'
            data_dict['Bowler'] = '-'

        # runout
        elif 'run' in score:
            data_dict['Batsman'] = " ".join(score[:score.index('run')]).strip()
            data_dict['Dismissal Type'] = 'Run out'
            data_dict['Fielder'] = " ".join(score[score.index('out')+1:]).strip()[1:-1]
            data_dict['Bowler'] = '-'

        # timed out
        elif 'timed' in score:
            data_dict['Batsman'] = " ".join(score[:score.index('timed')]).strip()
            data_dict['Dismissal Type'] = 'Timed out'
            data_dict['Fielder'] = '-'
            data_dict['Bowler'] = '-'

        # retd hurt
        elif 'retd' in score:
            data_dict['Batsman'] = " ".join(score[:score.index('retd')]).strip()
            data_dict['Dismissal Type'] = 'Retired hurt'
            data_dict['Fielder'] = '-'
            data_dict['Bowler'] = '-'

        # this is exceptional happend in this WC
        else:
            data_dict['Batsman'] = " ".join(score[:score.index('abs')]).strip()
            data_dict['Dismissal Type'] = 'Absent out'
            data_dict['Fielder'] = '-'
            data_dict['Bowler'] = '-'
        
        # Append the current data_dict to the data_list
        data_list.append(data_dict)

batsman_stats(batting_df['Score'])

# Create a DataFrame
scores_df = pd.DataFrame(data_list)

In [71]:
scores_df.head()

,Batsman,Dismissal Type,Fielder,Bowler
0,Bairstow,Catch,Daryl Mitchell,Santner
1,Malan,Catch,Latham,Matt Henry
2,Root,Bowled,-,Glenn Phillips
3,Harry Brook,Catch,Conway,Rachin Ravindra
4,Moeen,Bowled,-,Glenn Phillips


In [72]:
batting_stats.head()

,Runs,Balls,Boundaries,Sixes,StrikeRate
0,33,35,4,1,94.29
1,14,24,2,0,58.33
2,77,86,4,1,89.53
3,25,16,4,1,156.25
4,11,17,1,0,64.71


In [73]:
# Combining all dataframes along row
final_batting_df = pd.concat([batting_df,scores_df,batting_stats],axis=1)
final_batting_df

,Match_id,Score,Team,Opposite_Team,Innings,Batsman,Dismissal Type,Fielder,Bowler,Runs,Balls,Boundaries,Sixes,StrikeRate
0,1,Bairstow c Daryl Mitchell b Santner 33 35 ...,England,New Zealand,1,Bairstow,Catch,Daryl Mitchell,Santner,33,35,4,1,94.29
1,1,Malan c Latham b Matt Henry 14 24 2 0 58.33,England,New Zealand,1,Malan,Catch,Latham,Matt Henry,14,24,2,0,58.33
2,1,Root b Glenn Phillips 77 86 4 1 89.53,England,New Zealand,1,Root,Bowled,-,Glenn Phillips,77,86,4,1,89.53
3,1,Harry Brook c Conway b Rachin Ravindra 25 ...,England,New Zealand,1,Harry Brook,Catch,Conway,Rachin Ravindra,25,16,4,1,156.25
4,1,Moeen b Glenn Phillips 11 17 1 0 64.71,England,New Zealand,1,Moeen,Bowled,-,Glenn Phillips,11,17,1,0,64.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,48,Travis Head c Shubman Gill b Siraj 137 120...,Australia,India,2,Travis Head,Catch,Shubman Gill,Siraj,137,120,15,4,114.17
872,48,Mitchell Marsh c Rahul b Bumrah 15 15 1 1 ...,Australia,India,2,Mitchell Marsh,Catch,Rahul,Bumrah,15,15,1,1,100.00
873,48,Steven Smith lbw b Bumrah 4 9 1 0 44.44,Australia,India,2,Steven Smith,lbw,-,Bumrah,4,9,1,0,44.44
874,48,Marnus Labuschagne not out 58 110 4 0 52.73,Australia,India,2,Marnus Labuschagne,Not out,-,-,58,110,4,0,52.73


In [74]:
final_batting_df.drop(columns='Score',inplace=True) # Score column not needed 

In [75]:
final_batting_df.head()

,Match_id,Team,Opposite_Team,Innings,Batsman,Dismissal Type,Fielder,Bowler,Runs,Balls,Boundaries,Sixes,StrikeRate
0,1,England,New Zealand,1,Bairstow,Catch,Daryl Mitchell,Santner,33,35,4,1,94.29
1,1,England,New Zealand,1,Malan,Catch,Latham,Matt Henry,14,24,2,0,58.33
2,1,England,New Zealand,1,Root,Bowled,-,Glenn Phillips,77,86,4,1,89.53
3,1,England,New Zealand,1,Harry Brook,Catch,Conway,Rachin Ravindra,25,16,4,1,156.25
4,1,England,New Zealand,1,Moeen,Bowled,-,Glenn Phillips,11,17,1,0,64.71


In [76]:
# Data Inspection 
batting_df.shape,batting_stats.shape # Match found

((876, 5), (876, 5))

# Bowling Data Cleaning

In [77]:
bowling_df = pd.read_csv(r'F:\DA Projects\CWC2023\ICC_CWC_2023_Data_Analysis\Data\pre_bowling_df.csv')
bowling_df.head()

,Unnamed: 0,Match_id,Bowling,Team,Opposite_Team,Innings
0,0,1,Boult 10 1 48 1 0 1 4.80,New Zealand,England,1
1,1,1,Matt Henry 10 1 48 3 0 0 4.80,New Zealand,England,1
2,2,1,Santner 10 0 37 2 0 1 3.70,New Zealand,England,1
3,3,1,Neesham 7 0 56 0 0 4 8.00,New Zealand,England,1
4,4,1,Rachin Ravindra 10 0 76 1 0 0 7.60,New Zealand,England,1


In [78]:
bowling_df.drop(columns=['Unnamed: 0'],inplace=True)

In [79]:
def bowler_stats_extraction(samples):
    '''
    This Function will Extract Stats of Bowler Statitcs such as Overs,Runs,Maindens,Wickets and Economy (Numericals)
    Input : Bowler Stats as a row 
    Output : Bowler Stats as df 

    Sample Input : 'Boult   10 1 48 1 0 1 4.80'
    '''
    
    #for i in samples:
    bowling_stats = samples.split(' ')[-7:]
    Name = " ".join(samples.split(' ')[:-7]).strip()
    return pd.Series({
            'Bowler': Name,
            'Overs' : bowling_stats[0],
            'Maindens' : bowling_stats[1],
            'Runs' : bowling_stats[2],
            'Wickets' : bowling_stats[3],
            'No_Balls' : bowling_stats[4],
            'Wides' : bowling_stats[5],
            'Economy' : bowling_stats[6]})

In [80]:
bowl_stats = bowling_df['Bowling'].apply(bowler_stats_extraction)
bowl_stats.head(3)

,Bowler,Overs,Maindens,Runs,Wickets,No_Balls,Wides,Economy
0,Boult,10,1,48,1,0,1,4.80
1,Matt Henry,10,1,48,3,0,0,4.80
2,Santner,10,0,37,2,0,1,3.70


In [81]:
# Concatenating 2 dfs row_wise to get exact score card (Dismissal+Stats)
final_bowling_df = pd.concat([bowling_df,bowl_stats],axis=1)

In [82]:
final_bowling_df.drop(columns='Bowling',inplace=True)

In [83]:
final_bowling_df.head()

,Match_id,Team,Opposite_Team,Innings,Bowler,Overs,Maindens,Runs,Wickets,No_Balls,Wides,Economy
0,1,New Zealand,England,1,Boult,10,1,48,1,0,1,4.80
1,1,New Zealand,England,1,Matt Henry,10,1,48,3,0,0,4.80
2,1,New Zealand,England,1,Santner,10,0,37,2,0,1,3.70
3,1,New Zealand,England,1,Neesham,7,0,56,0,0,4,8.00
4,1,New Zealand,England,1,Rachin Ravindra,10,0,76,1,0,0,7.60


### Names Normalization 

In [84]:
# England Replacement
final_batting_df.replace({
    'Bairstow': 'Jonny Bairstow',
    'Malan': 'Dawid Malan',
    'Root': 'Joe Root',
    'Harry Brook': 'Harry Brook',  # No change in this case
    'Moeen': 'Moeen Ali',
    'Jos Buttler (c & wk)': 'Jos Buttler (C)',
    'Livingstone': 'Liam Livingstone',
    'Sam Curran': 'Sam Curran',
    'Chris Woakes': 'Chris Woakes',
    'Adil Rashid': 'Adil Rashid',
    'Mark Wood': 'Mark Wood',
    'R Topley': 'Reece Topley',
    'Stokes': 'Ben Stokes',
    'Willey': 'David Willey',
    'Gus Atkinson': 'Gus Atkinson'
},inplace=True)

# England Replacement
final_bowling_df.replace(
    {'Chris Woakes': 'Chris Woakes',
    'R Topley': 'Reece Topley',
    'Sam Curran': 'Sam Curran',
    'Adil Rashid': 'Adil Rashid',
    'Mark Wood': 'Mark Wood',
    'Livingstone': 'Liam Livingstone',
    'Root': 'Joe Root',
    'Willey': 'David Willey',
    'Gus Atkinson': 'Gus Atkinson',
    'Moeen': 'Moeen Ali',
    'Moeen Ali': 'Moeen Ali',
    'Liam Livingstone': 'Liam Livingstone',
    'Reece Topley': 'Reece Topley',
    'David Willey': 'David Willey'
},inplace=True)

In [85]:
# Pakistan Replacement
final_bowling_df
final_batting_df.replace({
    'Fakhar Zaman': 'Fakhar Zaman',
    'Imam-ul-Haq': 'Imam-ul-Haq',
    'Babar Azam (c)': 'Babar Azam (c)',
    'Rizwan (wk)': 'Mohammad Rizwan',
    'Saud Shakeel': 'Saud Shakeel',
    'Iftikhar Ahmed': 'Iftikhar Ahmed',
    'Mohammad Nawaz': 'Mohammad Nawaz',
    'Shadab Khan': 'Shadab Khan',
    'Hasan Ali': 'Hasan Ali',
    'Shaheen Afridi': 'Shaheen Afridi',
    'Haris Rauf': 'Haris Rauf',
    'Shafique': 'Shafique',
    'Mohammad Wasim Jr': 'Mohammad Wasim Jr',
    'Abdullah Shafique': 'Abdullah Shafique',
    'Mohammad Rizwan (wk)': 'Mohammad Rizwan',
    'Usama Mir': 'Usama Mir',
    'Agha Salman': 'Agha Salman'
},inplace=True)
final_bowling_df.replace({
    'Shaheen Afridi': 'Shaheen Afridi',
    'Hasan Ali': 'Hasan Ali',
    'Mohammad Nawaz': 'Mohammad Nawaz',
    'Haris Rauf': 'Haris Rauf',
    'Shadab Khan': 'Shadab Khan',
    'Iftikhar Ahmed': 'Iftikhar Ahmed',
    'Usama Mir': 'Usama Mir',
    'Mohammad Wasim Jr': 'Mohammad Wasim Jr',
    'Agha Salman': 'Agha Salman'
},inplace=True)

In [86]:
# Afghanistan Replacement
final_bowling_df
final_batting_df.replace({
    'Gurbaz (wk)': 'Rahmanullah Gurbaz',
    'Ibrahim Zadran': 'Ibrahim Zadran',
    'Rahmat': 'Rahmat Shah',
    'Shahidi (c)': 'Hashmatullah Shahidi (c)',
    'Najibullah': 'Najibullah Zadran',
    'Nabi': 'Mohammad Nabi',
    'Azmatullah': 'Azmatullah Omarzai',
    'Rashid Khan': 'Rashid Khan',
    'Mujeeb': 'Mujeeb Ur Rahman',
    'Naveen-ul-Haq': 'Naveen-ul-Haq',
    'Fazalhaq Farooqi': 'Fazalhaq Farooqi',
    'Ikram Alikhil (wk)': 'Ikram Alikhil',
    'Noor Ahmad': 'Noor Ahmad',
    'Rahmanullah Gurbaz': 'Rahmanullah Gurbaz',
    'Rahmat Shah': 'Rahmat Shah',
    'Hashmatullah Shahidi (c)': 'Hashmatullah Shahidi',
    'Azmatullah Omarzai': 'Azmatullah Omarzai',
    'Mohammad Nabi': 'Mohammad Nabi',
    'Mujeeb Ur Rahman': 'Mujeeb Ur Rahman',
    'Rahmanullah Gurbaz (wk)': 'Rahmanullah Gurbaz'
},inplace=True)

final_bowling_df.replace({
    'Mujeeb': 'Mujeeb Ur Rahman',
    'Fazalhaq Farooqi': 'Fazalhaq Farooqi',
    'Naveen-ul-Haq': 'Naveen-ul-Haq',
    'Nabi': 'Mohammad Nabi',
    'Rashid Khan': 'Rashid Khan',
    'Azmatullah': 'Azmatullah Omarzai',
    'Noor Ahmad': 'Noor Ahmad',
    'Mujeeb Ur Rahman': 'Mujeeb Ur Rahman',
    'Mohammad Nabi': 'Mohammad Nabi',
    'Azmatullah Omarzai': 'Azmatullah Omarzai',
    'Rahmat Shah': 'Rahmat Shah'},inplace=True)

In [87]:
# South Africa Replacement

final_batting_df.replace({
    'de Kock': 'Quinton de Kock',
    'Bavuma (c)': 'Temba Bavuma',
    'van der Dussen': 'Rassie van der Dussen',
    'Markram': 'Aiden Markram',
    'Klaasen (wk)': 'Heinrich Klaasen',
    'Miller': 'David Miller',
    'Marco Jansen': 'Marco Jansen',
    'de Kock (wk)': 'Quinton de Kock',
    'Klaasen': 'Heinrich Klaasen',
    'Rabada': 'Kagiso Rabada',
    'Maharaj': 'Keshav Maharaj',
    'Reeza Hendricks': 'Reeza Hendricks',
    'Markram (c)': 'Aiden Markram',
    'Gerald Coetzee': 'Gerald Coetzee',
    'Shamsi': 'Tabraiz Shamsi',
    'Temba Bavuma (c)': 'Temba Bavuma',
    'Quinton de Kock (wk)': 'Quinton de Kock',
    'Rassie van der Dussen': 'Rassie van der Dussen',
    'Aiden Markram': 'Aiden Markram',
    'Heinrich Klaasen': 'Heinrich Klaasen',
    'David Miller': 'David Miller',
    'Keshav Maharaj': 'Keshav Maharaj',
    'Kagiso Rabada': 'Kagiso Rabada',
    'Lungi Ngidi': 'Lungi Ngidi',
    'Tabraiz Shamsi': 'Tabraiz Shamsi',
    'Andile Phehlukwayo': 'Andile Phehlukwayo'
},inplace=True)

final_batting_df.replace({
    'Lungi Ngidi': 'Lungi Ngidi',
    'Marco Jansen': 'Marco Jansen',
    'Rabada': 'Kagiso Rabada',
    'Gerald Coetzee': 'Gerald Coetzee',
    'Maharaj': 'Keshav Maharaj',
    'Markram': 'Aiden Markram',
    'Shamsi': 'Tabraiz Shamsi',
    'Phehlukwayo': 'Andile Phehlukwayo',
    'Kagiso Rabada': 'Kagiso Rabada',
    'Keshav Maharaj': 'Keshav Maharaj',
    'Tabraiz Shamsi': 'Tabraiz Shamsi',
    'Lizaad Williams': 'Lizaad Williams',
    'Aiden Markram': 'Aiden Markram'
},inplace=True)

In [88]:
# Australia Replacement

final_batting_df.replace({
    'Warner': 'David Warner',
    'Mitchell Marsh': 'Mitchell Marsh',
    'Steven Smith': 'Steven Smith',
    'Labuschagne': 'Marnus Labuschagne',
    'Maxwell': 'Glenn Maxwell',
    'Alex Carey (wk)': 'Alex Carey',
    'Green': 'Cameron Green',
    'Cummins (c)': 'Pat Cummins',
    'Starc': 'Mitchell Starc',
    'Zampa': 'Adam Zampa',
    'Hazlewood': 'Josh Hazlewood',
    'Stoinis': 'Marcus Stoinis',
    'Josh Inglis (wk)': 'Josh Inglis',
    'Head': 'Travis Head',
    'David Warner': 'David Warner',
    'Marnus Labuschagne': 'Marnus Labuschagne',
    'Glenn Maxwell': 'Glenn Maxwell',
    'Marcus Stoinis': 'Marcus Stoinis',
    'Mitchell Starc': 'Mitchell Starc',
    'Pat Cummins (c)': 'Pat Cummins',
    'Adam Zampa': 'Adam Zampa',
    'Josh Hazlewood': 'Josh Hazlewood',
    'Travis Head': 'Travis Head'
},inplace=True)

final_bowling_df.replace({
    'Starc': 'Mitchell Starc',
    'Hazlewood': 'Josh Hazlewood',
    'Maxwell': 'Glenn Maxwell',
    'Cummins (c)': 'Pat Cummins',
    'Zampa': 'Adam Zampa',
    'Mitchell Marsh': 'Mitchell Marsh',
    'Stoinis': 'Marcus Stoinis',
    'Head': 'Travis Head',
    'Abbott': 'Sean Abbott',
    'Mitchell Starc': 'Mitchell Starc',
    'Josh Hazlewood': 'Josh Hazlewood',
    'Pat Cummins (c)': 'Pat Cummins',
    'Glenn Maxwell': 'Glenn Maxwell',
    'Cameron Green': 'Cameron Green',
    'Adam Zampa': 'Adam Zampa',
    'Marcus Stoinis': 'Marcus Stoinis',
    'Travis Head': 'Travis Head'
},inplace=True)

In [89]:
# Newzeland Replacement
final_bowling_df
final_batting_df.replace({
    'Conway': 'Devon Conway',
    'Will Young': 'Will Young',
    'Rachin Ravindra': 'Rachin Ravindra',
    'Daryl Mitchell': 'Daryl Mitchell',
    'Latham (c & wk)': 'Tom Latham',
    'Glenn Phillips': 'Glenn Phillips',
    'Chapman': 'Mark Chapman',
    'Santner': 'Mitchell Santner',
    'Matt Henry': 'Matt Henry',
    'Lockie Ferguson': 'Lockie Ferguson',
    'Boult': 'Trent Boult',
    'Williamson (c)': 'Kane Williamson',
    'Latham (wk)': 'Tom Latham',
    'Devon Conway': 'Devon Conway',
    'Kane Williamson (c)': 'Kane Williamson',
    'Tom Latham (c & wk)': 'Tom Latham',
    'James Neesham': 'James Neesham',
    'Mitchell Santner': 'Mitchell Santner',
    'Trent Boult': 'Trent Boult',
    'Tim Southee': 'Tim Southee',
    'Mark Chapman': 'Mark Chapman',
    'Tom Latham (wk)': 'Tom Latham'
},inplace=True)

final_bowling_df.replace({
    'Boult': 'Trent Boult',
    'Matt Henry': 'Matt Henry',
    'Santner': 'Mitchell Santner',
    'Neesham': 'Jimmy Neesham',
    'Rachin Ravindra': 'Rachin Ravindra',
    'Glenn Phillips': 'Glenn Phillips',
    'Lockie Ferguson': 'Lockie Ferguson',
    'Daryl Mitchell': 'Daryl Mitchell',
    'Southee': 'Tim Southee',
    'Trent Boult': 'Trent Boult',
    'Mitchell Santner': 'Mitchell Santner',
    'Tim Southee': 'Tim Southee',
    'Ish Sodhi': 'Ish Sodhi'
}
,inplace=True)

In [90]:
# Sri lanka Replacement

final_batting_df.replace({
    'Pathum Nissanka': 'Pathum Nissanka',
    'Kusal Perera': 'Kusal Perera',
    'Kusal Mendis': 'Kusal Mendis',
    'Samarawickrama (wk)': 'Sadeera Samarawickrama',
    'Asalanka': 'Charith Asalanka',
    'Dhananjaya de Silva': 'Dhananjaya de Silva',
    'Shanaka (c)': 'Dasun Shanaka',
    'Dunith Wellalage': 'Dunith Wellalage',
    'M Theekshana': 'Maheesh Theekshana',
    'Matheesha Pathirana': 'Matheesha Pathirana',
    'Kusal Mendis (c & wk)': 'Kusal Mendis',
    'Samarawickrama': 'Sadeera Samarawickrama',
    'C Karunaratne': 'Dimuth Karunaratne',
    'Lahiru Kumara': 'Lahiru Kumara',
    'Dilshan Madushanka': 'Dilshan Madushanka',
    'Karunaratne': 'Dimuth Karunaratne',
    'Mathews': 'Angelo Mathews',
    'Chameera': 'Dushmantha Chameera',
    'Rajitha': 'Kasun Rajitha',
    'Kusal Mendis (wk)': 'Kusal Mendis',
    'Sadeera Samarawickrama': 'Sadeera Samarawickrama',
    'Charith Asalanka': 'Charith Asalanka',
    'Dasun Shanaka (c)': 'Dasun Shanaka',
    'Kasun Rajitha': 'Kasun Rajitha',
    'Dushan Hemantha': 'Dushan Hemantha',
    'Dimuth Karunaratne': 'Dimuth Karunaratne',
    'Angelo Mathews': 'Angelo Mathews',
    'Dushmantha Chameera': 'Dushmantha Chameera',
    'Maheesh Theekshana': 'Maheesh Theekshana'
},inplace=True)
final_bowling_df.replace({
    'Rajitha': 'Lahiru Kumara',
    'Dilshan Madushanka': 'Dilshan Madushanka',
    'Shanaka (c)': 'Dasun Shanaka',
    'Dhananjaya de Silva': 'Dhananjaya de Silva',
    'Matheesha Pathirana': 'Matheesha Pathirana',
    'Dunith Wellalage': 'Dunith Wellalage',
    'C Karunaratne': 'Dimuth Karunaratne',
    'M Theekshana': 'Maheesh Theekshana',
    'Dushan Hemantha': 'Dushan Hemantha',
    'Mathews': 'Angelo Mathews',
    'Lahiru Kumara': 'Lahiru Kumara',
    'Chameera': 'Dushmantha Chameera',
    'Maheesh Theekshana': 'Maheesh Theekshana',
    'Dasun Shanaka (c)': 'Dasun Shanaka',
    'Charith Asalanka': 'Charith Asalanka',
    'Chamika Karunaratne': 'Chamika Karunaratne',
    'Kasun Rajitha': 'Kasun Rajitha',
    'Angelo Mathews': 'Angelo Mathews',
    'Dushmantha Chameera': 'Dushmantha Chameera'
},inplace=True)

In [91]:
# Bangladesh Replacement
final_batting_df.replace({
    'Litton Das': 'Litton Das',
    'Tanzid Hasan': 'Tanzid Hasan',
    'Mehidy Hasan Miraz': 'Mehidy Hasan Miraz',
    'Shanto': 'Najmul Hossain Shanto',
    'Shakib (c)': 'Shakib Al Hasan',
    'Mushfiqur Rahim (wk)': 'Mushfiqur Rahim',
    'Towhid Hridoy': 'Towhid Hridoy',
    'Mahmudullah': 'Mahmudullah',
    'Taskin Ahmed': 'Taskin Ahmed',
    'Mustafizur': 'Mustafizur Rahman',
    'Shoriful Islam': 'Shoriful Islam',
    'Shanto (c)': 'Najmul Hossain Shanto',
    'Nasum Ahmed': 'Nasum Ahmed',
    'Mahedi Hasan': 'Mahedi Hasan',
    'Najmul Hossain Shanto': 'Najmul Hossain Shanto',
    'Shakib Al Hasan (c)': 'Shakib Al Hasan',
    'Mustafizur Rahman': 'Mustafizur Rahman',
    'Hasan Mahmud': 'Hasan Mahmud',
    'Tanzim Hasan Sakib': 'Tanzim Hasan Sakib'
},inplace=True)

final_bowling_df.replace({
    'Taskin Ahmed': 'Taskin Ahmed',
    'Shoriful Islam': 'Shoriful Islam',
    'Mustafizur': 'Mustafizur Rahman',
    'Shakib (c)': 'Shakib Al Hasan',
    'Mehidy Hasan Miraz': 'Mehidy Hasan Miraz',
    'Mahmudullah': 'Mahmudullah',
    'Mahedi Hasan': 'Mahedi Hasan',
    'Hasan Mahmud': 'Hasan Mahmud',
    'Nasum Ahmed': 'Nasum Ahmed',
    'Tanzim Hasan Sakib': 'Tanzim Hasan Sakib',
    'Mustafizur Rahman': 'Mustafizur Rahman',
    'Shakib Al Hasan (c)': 'Shakib Al Hasan',
    'Najmul Hossain Shanto': 'Najmul Hossain Shanto'
},inplace=True)

In [92]:
#Netherlands Replacement
final_batting_df.replace({
    'Vikramjit Singh': 'Vikramjit Singh',
    'Max ODowd': 'Max ODowd',
    'Ackermann': 'Colin Ackermann',
    'Bas de Leede': 'Bas de Leede',
    'Sybrand Engelbrecht': 'Sybrand Engelbrecht',
    'Teja Nidamanuru': 'Teja Nidamanuru',
    'Edwards (c & wk)': 'Scott Edwards',
    'van Beek': 'Logan van Beek',
    'van der Merwe': 'Roelof van der Merwe',
    'Aryan Dutt': 'Aryan Dutt',
    'van Meekeren': 'Paul van Meekeren',
    'Barresi': 'Wesley Barresi',
    'Shariz Ahmad': 'Shariz Ahmad',
    'Saqib Zulfiqar': 'Saqib Zulfiqar',
    'Colin Ackermann': 'Colin Ackermann',
    'Scott Edwards (c & wk)': 'Scott Edwards',
    'Roelof van der Merwe': 'Roelof van der Merwe',
    'Logan van Beek': 'Logan van Beek',
    'Paul van Meekeren': 'Paul van Meekeren',
    'Ryan Klein': 'Ryan Klein',
    'Wesley Barresi': 'Wesley Barresi'
},inplace=True)


final_bowling_df.replace({
    'Aryan Dutt': 'Aryan Dutt',
    'van Beek': 'Logan van Beek',
    'Ackermann': 'Colin Ackermann',
    'van Meekeren': 'Paul van Meekeren',
    'Bas de Leede': 'Bas de Leede',
    'van der Merwe': 'Roelof van der Merwe',
    'Vikramjit Singh': 'Vikramjit Singh',
    'Saqib Zulfiqar': 'Saqib Zulfiqar',
    'Ryan Klein': 'Ryan Klein',
    'Logan van Beek': 'Logan van Beek',
    'Colin Ackermann': 'Colin Ackermann',
    'Paul van Meekeren': 'Paul van Meekeren',
    'Roelof van der Merwe': 'Roelof van der Merwe',
    'Shariz Ahmad': 'Shariz Ahmad'
},inplace=True)

In [93]:
# India Replacement
final_batting_df.replace({
    'Rohit (c)': 'Rohit Sharma',
    'Shubman Gill': 'Shubman Gill',
    'Kohli': 'Virat Kohli',
    'Shreyas Iyer': 'Shreyas Iyer',
    'Rahul (wk)': 'KL Rahul',
    'Suryakumar Yadav': 'Suryakumar Yadav',
    'Ravindra Jadeja': 'Ravindra Jadeja',
    'Shami': 'Mohammed Shami',
    'Bumrah': 'Jasprit Bumrah',
    'Kuldeep Yadav': 'Kuldeep Yadav',
    'Siraj': 'Mohammed Siraj',
    'Rohit Sharma (c)': 'Rohit Sharma',
    'Ishan Kishan': 'Ishan Kishan',
    'Virat Kohli': 'Virat Kohli',
    'KL Rahul (wk)': 'KL Rahul',
    'Hardik Pandya': 'Hardik Pandya',
    'Mohammed Shami': 'Mohammed Shami',
    'Ashwin': 'Ravichandran Ashwin'
},inplace=True)

final_bowling_df.replace({
    'Bumrah': 'Jasprit Bumrah',
    'Siraj': 'Mohammed Siraj',
    'Hardik Pandya': 'Hardik Pandya',
    'Ashwin': 'Ravichandran Ashwin',
    'Kuldeep Yadav': 'Kuldeep Yadav',
    'Ravindra Jadeja': 'Ravindra Jadeja',
    'Thakur': 'Shardul Thakur',
    'Kohli': 'Virat Kohli',
    'Shami': 'Mohammed Shami',
    'Jasprit Bumrah': 'Jasprit Bumrah',
    'Mohammed Siraj': 'Mohammed Siraj',
    'Mohammed Shami': 'Mohammed Shami',
    'Virat Kohli': 'Virat Kohli',
    'Shubman Gill': 'Shubman Gill',
    'Suryakumar Yadav': 'Suryakumar Yadav',
    'Rohit Sharma (c)': 'Rohit Sharma'
},inplace=True)

In [94]:
# Bowler Names Normalization
final_bowling_df.replace({
    'Trent Boult': 'Trent Boult',
    'Matt Henry': 'Matt Henry',
    'Mitchell Santner': 'Mitchell Santner',
    'Jimmy Neesham': 'Jimmy Neesham',
    'Rachin Ravindra': 'Rachin Ravindra',

    'Aryan Dutt': 'Aryan Dutt',
    'Logan van Beek': 'Logan van Beek',
    'Colin Ackermann': 'Colin Ackermann',
    'Paul van Meekeren': 'Paul van Meekeren',
    'Bas de Leede': 'Bas de Leede',
    'Roelof van der Merwe': 'Roelof van der Merwe',
    'Vikramjit Singh': 'Vikramjit Singh',
    'Saqib Zulfiqar': 'Saqib Zulfiqar',
    'Ryan Klein': 'Ryan Klein',
    'Shariz Ahmad': 'Shariz Ahmad',

    'Taskin Ahmed': 'Taskin Ahmed',
    'Shoriful Islam': 'Shoriful Islam',
    'Mustafizur Rahman': 'Mustafizur Rahman',
    'Shakib Al Hasan': 'Shakib Al Hasan',
    'Mehidy Hasan Miraz': 'Mehidy Hasan Miraz',
    'Mahmudullah': 'Mahmudullah',
    'Mahedi Hasan': 'Mahedi Hasan',
    'Hasan Mahmud': 'Hasan Mahmud',
    'Nasum Ahmed': 'Nasum Ahmed',
    'Tanzim Hasan Sakib': 'Tanzim Hasan Sakib',
    'Najmul Hossain Shanto': 'Najmul Hossain Shanto',

    'Lahiru Kumara': 'Lahiru Kumara',
    'Dilshan Madushanka': 'Dilshan Madushanka',
    'Dasun Shanaka': 'Dasun Shanaka',
    'Dhananjaya de Silva': 'Dhananjaya de Silva',
    'Matheesha Pathirana': 'Matheesha Pathirana',
    'Dunith Wellalage': 'Dunith Wellalage',
    'Dimuth Karunaratne': 'Dimuth Karunaratne',
    'Maheesh Theekshana': 'Maheesh Theekshana',
    'Dushan Hemantha': 'Dushan Hemantha',
    'Angelo Mathews': 'Angelo Mathews',
    'Dushmantha Chameera': 'Dushmantha Chameera',
    'Charith Asalanka': 'Charith Asalanka',
    'Chamika Karunaratne': 'Chamika Karunaratne',
    'Kasun Rajitha': 'Kasun Rajitha',

    'Jasprit Bumrah': 'Jasprit Bumrah',
    'Mohammed Siraj': 'Mohammed Siraj',
    'Hardik Pandya': 'Hardik Pandya',
    'Ravichandran Ashwin': 'Ravichandran Ashwin',
    'Kuldeep Yadav': 'Kuldeep Yadav',
    'Ravindra Jadeja': 'Ravindra Jadeja',
    'Shardul Thakur': 'Shardul Thakur',
    'Virat Kohli': 'Virat Kohli',
    'Mohammed Shami': 'Mohammed Shami',
    'Shubman Gill': 'Shubman Gill',
    'Suryakumar Yadav': 'Suryakumar Yadav',
    'Rohit Sharma': 'Rohit Sharma',

    'Shaheen Afridi': 'Shaheen Afridi',
    'Hasan Ali': 'Hasan Ali',
    'Mohammad Nawaz': 'Mohammad Nawaz',
    'Haris Rauf': 'Haris Rauf',
    'Shadab Khan': 'Shadab Khan',
    'Iftikhar Ahmed': 'Iftikhar Ahmed',
    'Usama Mir': 'Usama Mir',
    'Mohammad Wasim Jr': 'Mohammad Wasim Jr',
    'Agha Salman': 'Agha Salman',

    'Mitchell Starc': 'Mitchell Starc',
    'Josh Hazlewood': 'Josh Hazlewood',
    'Glenn Maxwell': 'Glenn Maxwell',
    'Pat Cummins': 'Pat Cummins',
    'Adam Zampa': 'Adam Zampa',
    'Mitchell Marsh': 'Mitchell Marsh',
    'Marcus Stoinis': 'Marcus Stoinis',
    'Travis Head': 'Travis Head',
    'Sean Abbott': 'Sean Abbott',
    'Cameron Green': 'Cameron Green',

    'Chris Woakes': 'Chris Woakes',
    'Reece Topley': 'Reece Topley',
    'Sam Curran': 'Sam Curran',
    'Adil Rashid': 'Adil Rashid',
    'Mark Wood': 'Mark Wood',
    'Liam Livingstone': 'Liam Livingstone',
    'Joe Root': 'Joe Root',
    'David Willey': 'David Willey',
    'Gus Atkinson': 'Gus Atkinson',
    'Moeen Ali': 'Moeen Ali',
    'Lungi Ngidi': 'Lungi Ngidi',
    'Marco Jansen': 'Marco Jansen',
    'Rabada': 'Kagiso Rabada',
    'Gerald Coetzee': 'Gerald Coetzee',
    'Maharaj': 'Keshav Maharaj',
    'Markram': 'Aiden Markram',
    'Shamsi': 'Tabraiz Shamsi',
    'Phehlukwayo': 'Phehlukwayo',
    'Kagiso Rabada': 'Kagiso Rabada',
    'Keshav Maharaj': 'Keshav Maharaj',
    'Tabraiz Shamsi': 'Tabraiz Shamsi',
    'Lizaad Williams': 'Lizaad Williams',
    'Mujeeb Ur Rahman': 'Mujeeb Ur Rahman',
    'Fazalhaq Farooqi': 'Fazalhaq Farooqi',
    'Naveen-ul-Haq': 'Naveen-ul-Haq',
    'Mohammad Nabi': 'Mohammad Nabi',
    'Rashid Khan': 'Rashid Khan',
    'Azmatullah Omarzai': 'Azmatullah Omarzai',
    'Noor Ahmad': 'Noor Ahmad',
    'Rahmat Shah': 'Rahmat Shah'},inplace=True)

In [95]:
# Creating bowler type column
bowler_types = {
    'Trent Boult': 'Left-arm Seamer',
    'Matt Henry': 'Right-arm Fast Medium',
    'Mitchell Santner': 'Left-arm Orthodox Spin',
    'Jimmy Neesham': 'Right-arm Medium-fast',
    'Rachin Ravindra': 'Left-arm Orthodox Spin',
    'Aryan Dutt': 'Right-arm Medium-fast',
    'Logan van Beek': 'Right-arm Medium-fast',
    'Colin Ackermann': 'Right-arm Off Spin',
    'Paul van Meekeren': 'Right-arm Medium-fast',
    'Bas de Leede': 'Right-arm Medium-fast',
    'Roelof van der Merwe': 'Left-arm Orthodox Spin',
    'Vikramjit Singh': 'Right-arm Medium-fast',
    'Saqib Zulfiqar': 'Left-arm Orthodox Spin',
    'Ryan Klein': 'Right-arm Medium-fast',
    'Shariz Ahmad': 'Left-arm Orthodox Spin',
    'Taskin Ahmed': 'Right-arm Fast',
    'Shoriful Islam': 'Left-arm Fast',
    'Mustafizur Rahman': 'Left-arm Fast',
    'Shakib Al Hasan': 'Left-arm Orthodox Spin',
    'Mehidy Hasan Miraz': 'Right-arm Off Spin',
    'Mahmudullah': 'Right-arm Off Spin',
    'Mahedi Hasan': 'Right-arm Leg Spin',
    'Hasan Mahmud': 'Right-arm Fast',
    'Nasum Ahmed': 'Left-arm Orthodox Spin',
    'Tanzim Hasan Sakib': 'Right-arm Medium-fast',
    'Najmul Hossain Shanto': 'Right-arm Off Spin',
    'Lahiru Kumara': 'Right-arm Fast',
    'Dilshan Madushanka': 'Right-arm Fast',
    'Dasun Shanaka': 'Right-arm Leg Spin',
    'Dhananjaya de Silva': 'Right-arm Off Spin',
    'Matheesha Pathirana': 'Right-arm Fast',
    'Dunith Wellalage': 'Right-arm Leg Spin',
    'Dimuth Karunaratne': 'Right-arm Medium',
    'Maheesh Theekshana': 'Right-arm Leg Spin',
    'Dushan Hemantha': 'Right-arm Leg Spin',
    'Angelo Mathews': 'Right-arm Fast Medium',
    'Dushmantha Chameera': 'Right-arm Fast',
    'Charith Asalanka': 'Right-arm Leg Spin',
    'Chamika Karunaratne': 'Right-arm Medium-fast',
    'Kasun Rajitha': 'Right-arm Fast',
    'Jasprit Bumrah': 'Right-arm Fast',
    'Mohammed Siraj': 'Right-arm Fast',
    'Hardik Pandya': 'Right-arm Medium-fast',
    'Ravichandran Ashwin': 'Right-arm Off Spin',
    'Kuldeep Yadav': 'Left-arm Chinaman',
    'Ravindra Jadeja': 'Left-arm Orthodox Spin',
    'Shardul Thakur': 'Right-arm Medium-fast',
    'Virat Kohli': 'Right-arm Medium',
    'Mohammed Shami': 'Right-arm Fast',
    'Shubman Gill': 'Right-arm Leg Spin',
    'Suryakumar Yadav': 'Right-arm Leg Spin',
    'Rohit Sharma': 'Right-arm Off Spin',
    'Shaheen Afridi': 'Left-arm Fast',
    'Hasan Ali': 'Right-arm Fast',
    'Mohammad Nawaz': 'Left-arm Orthodox Spin',
    'Haris Rauf': 'Right-arm Fast',
    'Shadab Khan': 'Right-arm Leg Spin',
    'Iftikhar Ahmed': 'Right-arm Off Spin',
    'Usama Mir': 'Right-arm Leg Spin',
    'Mohammad Wasim Jr': 'Right-arm Fast',
    'Agha Salman': 'Right-arm Leg Spin',
    'Mitchell Starc': 'Left-arm Fast',
    'Josh Hazlewood': 'Right-arm Fast Medium',
    'Glenn Maxwell': 'Right-arm Off Spin',
    'Pat Cummins': 'Right-arm Fast',
    'Adam Zampa': 'Right-arm Leg Spin',
    'Mitchell Marsh': 'Right-arm Medium-fast',
    'Marcus Stoinis': 'Right-arm Medium-fast',
    'Travis Head': 'Right-arm Leg Spin',
    'Sean Abbott': 'Right-arm Medium-fast',
    'Cameron Green': 'Right-arm Medium-fast',
    'Chris Woakes': 'Right-arm Fast Medium',
    'Reece Topley': 'Left-arm Medium-fast',
    'Sam Curran': 'Left-arm Medium-fast',
    'Adil Rashid': 'Leg Break',
    'Mark Wood': 'Right-arm Fast',
    'Liam Livingstone': 'Right-arm Leg Spin',
    'Joe Root': 'Right-arm Leg Spin',
    'David Willey': 'Left-arm Medium-fast',
    'Gus Atkinson': 'Right-arm Fast',
    'Moeen Ali': 'Right-arm Off Spin',
    'Lungi Ngidi': 'Right-arm Fast',
    'Marco Jansen': 'Left-arm Fast',
    'Rabada': 'Right-arm Fast',
    'Gerald Coetzee': 'Right-arm Fast',
    'Maharaj': 'Left-arm Orthodox Spin',
    'Markram': 'Right-arm Medium-fast',
    'Shamsi': 'Leg Break',
    'Phehlukwayo': 'Right-arm Medium-fast',
    'Kagiso Rabada': 'Right-arm Fast',
    'Keshav Maharaj': 'Left-arm Orthodox Spin',
    'Tabraiz Shamsi': 'Leg Break',
    'Lizaad Williams': 'Right-arm Fast',
    'Mujeeb Ur Rahman': 'Off Spin',
    'Fazalhaq Farooqi': 'Right-arm Fast',
    'Naveen-ul-Haq': 'Right-arm Fast',
    'Mohammad Nabi': 'Off Spin',
    'Rashid Khan': 'Leg Spin',
    'Azmatullah Omarzai': 'Right-arm Fast',
    'Noor Ahmad': 'Leg Spin',
    'Rahmat Shah': 'Off Spin',
    'Glenn Phillips': 'Leg Spin',
    'Lockie Ferguson': 'Right-arm Fast',
    'Daryl Mitchell': 'Right-arm Medium-fast',
    'Aiden Markram': 'Right-arm Leg Spin',
    'Rachin Ravindra': 'Left-arm Orthodox Spin',
    'Tim Southee': 'Right-arm Fast Medium',
    'Ish Sodhi': 'Leg Spin'
}

In [96]:
def bowler_type(bowler):
    return bowler_types.get(bowler)

final_bowling_df['Bowler_Type'] = final_bowling_df['Bowler'].apply(bowler_type)

In [97]:
final_bowling_df.head()

,Match_id,Team,Opposite_Team,Innings,Bowler,Overs,Maindens,Runs,Wickets,No_Balls,Wides,Economy,Bowler_Type
0,1,New Zealand,England,1,Trent Boult,10,1,48,1,0,1,4.80,Left-arm Seamer
1,1,New Zealand,England,1,Matt Henry,10,1,48,3,0,0,4.80,Right-arm Fast Medium
2,1,New Zealand,England,1,Mitchell Santner,10,0,37,2,0,1,3.70,Left-arm Orthodox Spin
3,1,New Zealand,England,1,Jimmy Neesham,7,0,56,0,0,4,8.00,Right-arm Medium-fast
4,1,New Zealand,England,1,Rachin Ravindra,10,0,76,1,0,0,7.60,Left-arm Orthodox Spin


In [99]:
final_bowling_df['Bowler_Type'].value_counts()

Bowler_Type
Right-arm Fast            157
Left-arm Orthodox Spin     66
Right-arm Medium-fast      66
Right-arm Leg Spin         60
Right-arm Off Spin         54
Left-arm Fast              44
Right-arm Fast Medium      35
Leg Spin                   24
Off Spin                   19
Leg Break                  13
Left-arm Medium-fast       12
Left-arm Chinaman          11
Left-arm Seamer            10
Right-arm Medium            3
Name: count, dtype: int64

In [100]:
# Capitan Names Normalization 
final_batting_df['Batsman'].replace({'Jos Buttler (C)':'Jos Buttler'},inplace=True)
final_batting_df['Batsman'].replace({'Babar Azam (c)':'Babar Azam'},inplace=True)
final_bowling_df.rename(columns={'Maindens':'Maidens'},inplace=True)

In [102]:
final_batting_df.to_csv(r'F:\DA Projects\CWC2023\ICC_CWC_2023_Data_Analysis\Data\final_batting_df.csv',index=False)

In [ ]:
final_bowling_df.to_csv(r'F:\DA Projects\CWC2023\ICC_CWC_2023_Data_Analysis\Data\final_bowling_df.csv',index=False)